# Extracting IPs from SERP and Watch pages

**September 30, 2021**  
- Basis provided by Eni Mustafaraj
- Edited by Zeynep Yalcin 

This notebook is about SERPs and Watch pages that were collected with the `requests` package, which does not render the YouTube data into HTML, but keeps them in a JSON format inside `<script>` tags.

There are multiple functions because the notebook can be used to do the following:
- extract IPs from SERP pages 
- extract IPs from Watch pages
- extract the list of videos from a SERP page

In addition to these functions, there is also test code that:
- collects all SERP pages for the Crowd audit list of queries (84 queries) and extracts their IPs (it found 25 pages with an IP)
- collects the Watch pages for the query "flat earth proof" and extracts their IPs (it found 18 pages with an IP)
- merges information from these two collection to create a dataframe for the "flat earh proof" videos. Such a dataframe can then be easily transformed to prepare the data for the analysis.

**Explanation of functions**

1. Helper function `extractIPinfo`: given a dictionary with the INFO_PANEL data, it will extract them and return the result in a dictionary. 
2. Helper function `findIPCR_SERP`: needed to go deep in the nested YouTube data and extract the InfoPanelContainerRenderer. It applies to SERP pages.
3. Helper function `findIPCR_Watch`: the same as function 2, but it applies to Watch pages.
4. Helper function `getYTdata`: It looks for a Javascript variable stored in a **`<script>`** tag, which has as its value a dictionary with all the data in the YouTube page (info panel and list of videos). Returns the string of the script. 
5. The main function `extractIP`: this is the function that is called with a filepath in order to search the HTML page to find the section with the INFO_PANEL data. By providing as a second argument the type of the page (SERP or Watch), it returns the corresponding IP panel, if present.
6. The main function `extractVideosFromSERP`: this is very similar to `extractIP`, but instead of extracting the IP info, it extract the list of all videos in the SERP page, storing their ID and title.


In [1]:
import requests
from bs4 import BeautifulSoup as BS
import json
import os

### Helper function `extractIPinfo`

This function allows us to extract the information of the IP, such as title, source, URL, and text.

In [2]:
def extractIPinfo(infoPanelSection):
    """Extract the information from the provided InfoPanel section.
    """
    try:
        title = infoPanelSection['infoPanelContentRenderer']['title']['runs'][0]['text']
        source = infoPanelSection['infoPanelContentRenderer']['source']['runs'][0]['text']
        url = infoPanelSection['infoPanelContentRenderer']['sourceEndpoint']['urlEndpoint']['url']
        text = infoPanelSection['infoPanelContentRenderer']['paragraphs'][0]['runs'][0]['text']
        return {'title': title, 'source': source, 'url': url, 'text': text}
    except:
        print("Couldn't extract information from the INFO_PANEL section.")
        return {}

### Helper function `findIPCR_SERP`

Find the InfoPanelContainerRenderer (IPCR) in the deeply nested YouTube data. It applies to SERP pages.

In [3]:
def findIPCR_SERP(jsonData):
    """Extract the particular dictionary that contains the InfoPanelContainer.
    This dictionary is nested really deep in the HTML/JSON page.
    This function works with SERP pages.
    """
    section = jsonData['contents']['twoColumnSearchResultsRenderer']\
                      ['primaryContents']['sectionListRenderer']\
                      ['contents'][0]['itemSectionRenderer']['contents']
        
    # the ipcr dict is not always first, so we have to search for it
    ipcr = [el for el in section if 'infoPanelContainerRenderer' in el][0]\
                      ['infoPanelContainerRenderer']['content']
    return ipcr

### Helper function `findIPCR_Watch`
Find the InfoPanelContainerRenderer (IPCR) in the deeply nested YouTube data. It applies to Watch pages.

In [4]:
def findIPCR_Watch(jsonData):
    """Extract the particular dictionary that contains the InfoPanelContainer.
    This dictionary is nested really deep.
    This function works with Watch pages.
    """
    section = jsonData['contents']['twoColumnWatchNextResults']\
                      ['results']['results']['contents'][0]\
                      ['itemSectionRenderer']['contents']
    
    ipcr = [el for el in section if 'infoPanelContainerRenderer' in el][0]\
           ['infoPanelContainerRenderer']['content']
    
    return ipcr

### Helper function `getYTData`

Gets the content of the variable `ytInitialData` from the HTML file. The content of this variable will be converted to a dictionary with the help of `json.loads` to extract the IPs and the videos (from the SERP page). However, this function returns a string, so that it's easy to search for substrings (such as "INFO_PANEL") in it.

In [5]:
def getYTData(filePath):
    """Helper function that extract the variable that contains all the data on
    the YouTube page.
    """
    # turn HTML into the DOM object
    dom = BS(open(filePath, encoding="utf8"), 'html.parser')
    
    # find all script tags
    scripts = dom.find_all('script')
    
    # find the script that contains the YouTube data (panel, video, etc.)
    for script in scripts:
        scriptText = script.string
        if scriptText and 'var ytInitialData' in scriptText:
            ourScript = scriptText
            break
    
    # the script text contains the variable name, we'll remove that
    start = ourScript.find('{') 
    if ourScript.endswith(';'):
        variableText = ourScript[start:-1]
    else:
        variableText = ourScript[start:]
        
    return variableText

### Main function `extractIP`

This function will call some of the helper functions listed above and return either a dictionary with the IP information or an empty dictionary.

In [6]:
def extractIP(filePath, pageType):
    """Function that returns the IP content of a YouTube SERP or 
    Watch page if it finds it, or an empty dictionary, if it doesn't.
    If pageType is 'SERP', extracts IP from SERP pages.
    If pageType is 'Watch' extracts IP from Watch pages.
    """
    
    ytDataString = getYTData(filePath)
    
    if 'INFO_PANEL_CONTAINER' not in ytDataString:
        return {}  # leave function if INFO_PANEL not found   
        
    # convert from string to dictionary
    jsonData = json.loads(ytDataString)
    
    # to extract INFO_PANEL, use the appropriate function
    funcNameDct = {'SERP': findIPCR_SERP, 'Watch': findIPCR_Watch}
    
    try:
        # first get the function name, then pass the argument
        ipcr = funcNameDct[pageType](jsonData)
        
        # call helper function to extract the detailed info from the panel
        return extractIPinfo(ipcr)
                
    except:
        print(filePath, "- Check your code for extracting the info panel!")
        return {}

### Main function `extractVideosFromSERP`

Similar to the previous function, but instead of looking for the INFO_PANEL, it returns a list of dictionaries with information about all the videos on the page.

In [7]:
def extractVideosFromSERP(filePath):
    """Given a SERP file, extracts and returns the list of all videos in the page.
    Each list entry is a simple dictionary with two keys: videoId and title.
    """
    
    ytDataString = getYTData(filePath)
    
     # convert from string to dictionary
    jsonData = json.loads(ytDataString)
    
    sectionList = jsonData['contents']['twoColumnSearchResultsRenderer']\
                      ['primaryContents']['sectionListRenderer']\
                      ['contents'][0]['itemSectionRenderer']['contents']
    
    # Find all videos, store videoId and title for each of them
    videosLst = []
    for entry in sectionList:
        if 'videoRenderer' in entry:
            videosLst.append({'videoId': entry['videoRenderer']['videoId'],
                             'title': entry['videoRenderer']['title']['runs'][0]['text']})

    return videosLst

## Step 1: Extracting IPs from SERP pages

I will collect SERPs for the queries of CROWD_DS, then pass them through the fuctions above to find out if they have IPs, and which ones.

In [8]:
with open('simple_queries_crowd.txt') as inputF:
    queries = [q.strip() for q in inputF.readlines()]
    
len(queries)

84

In [9]:
queries[5]

'illuminati conspiracy theory'

In [10]:
with open('simple_queries_audit.txt') as inputF:
    queries_audit = [q.strip() for q in inputF.readlines()]
    
len(queries_audit)

49

In the code below we extract the SERP data by sending a requestingt to youtube for each query that we have and create a folder to save them in the form of a URL. 

In [11]:
for q in queries:
    # send a request to YouTube
    response = requests.get("http://www.youtube.com/results?search_query={}".format("+".join(q.split())))
   
    # if successful, store the data into a file
    if response.status_code == 200:
        html = response.text
        with open('SERPS-crowd/{}.html'.format(q), 'w') as outF:
            outF.write(html)

In [12]:
for m in queries_audit:
    # send a request to YouTube
    response_audit = requests.get("http://www.youtube.com/results?search_query={}".format("+".join(m.split())))
   
    # if successful, store the data into a file
    if response_audit.status_code == 200:
        html_audit = response_audit.text
        with open('SERPS-audit/{}.html'.format(m), 'w') as outM:
            outM.write(html)

**Check that the files were written, we want the lengths to be 84 and 49** 

In [13]:
files = os.listdir('SERPS-crowd')
files = [f for f in files if not f.startswith('.')]

In [14]:
len(files)

84

In [15]:
files_audit = os.listdir('SERPS-audit')
files_audit = [m for m in files_audit if not m.startswith('.')]

In [16]:
len(files_audit)

49

In [17]:
#check contents 
files[10:14]

['coronavirus was man made.html',
 'flat earth real.html',
 'seth rich.html',
 'covid grew in lab.html']

In [18]:
#check contents 
files_audit[10:14]

['earth is flat.html',
 'Vaccines are toxic.html',
 'flat earth conspiracy.html',
 'flat earth brothers channel.html']



**Find the SERPs with IP**

Now we can use the functions provided  to extract IPs. With this function we are able to extarct the information panels in each file. 

In [19]:
queriesWithIPs = {}  # keep track of the INFO_PANEL data in a dict
withoutIPs = []      # keep track of SERPs without IP

for f in files:
    query = f.split('.html')[0]
    result = extractIP(os.path.join('SERPS-crowd', f), 'SERP')
    if result:
        queriesWithIPs[query] = result
    else:
        withoutIPs.append(query)
        
print('pages with IPs:', len(queriesWithIPs))
print('pages without IPs:', len(withoutIPs))

pages with IPs: 26
pages without IPs: 58


In [20]:
queriesWithIPs_audit = {}  # keep track of the INFO_PANEL data in a dict
withoutIPs_audit = []      # keep track of SERPs without IP

for m in files_audit:
    query = m.split('.html')[0]
    result_audit = extractIP(os.path.join('/Users/zeynoyalcin/Desktop/SERPS-audit', m), 'SERP')
    if result_audit:
        queriesWithIPs_audit[query] = result_audit
    else:
        withoutIPs_audit.append(query)
        
print('pages with IPs:', len(queriesWithIPs_audit))
print('pages without IPs:', len(withoutIPs_audit))

pages with IPs: 17
pages without IPs: 32


Let's see the names of the queries whose SERPs have IPs:

In [21]:
sorted(queriesWithIPs.keys())

['area 51',
 'barack obama born in kenya',
 'climate change a hoax',
 'climate change fake',
 "climate change isn't real",
 'climate change not real',
 'flat earth',
 'flat earth proof',
 'flat earth real',
 'flat earth theory',
 'global warming',
 'global warming not real',
 'holocaust fake',
 'illuminati',
 'illuminati conspiracy theory',
 'jfk assassination',
 'jfk assassination conspiracy',
 'jfk assassination real killer',
 'jfk killer',
 'moon landing not real',
 'obama birth certificate',
 'pizza gate',
 'pizzagate',
 'princess diana death',
 'qanon',
 'russia shot down mh370']

In [22]:
sorted(queriesWithIPs_audit.keys())

['Chemtrail',
 'Chemtrail 2018',
 'Chemtrail flu',
 'Chemtrails',
 'chemtrail documentary',
 'chemtrail pilot',
 'chemtrail pilot speaks out',
 'chemtrail plane up close',
 'chemtrail whistleblower',
 'chemtrails exposed',
 'earth is flat',
 'flat earth',
 'flat earth british',
 'flat earth brothers channel',
 'flat earth conspiracy',
 'flat earth paradise',
 'flat earth proof']

We can look at the IP info of some of the queries:

In [23]:
queriesWithIPs['qanon']

{'title': 'QAnon',
 'source': 'Wikipedia',
 'url': 'https://en.wikipedia.org/wiki/QAnon?wprov=yicw1',
 'text': 'QAnon is a disproven far-right conspiracy theory alleging that a cabal of Satanic, cannibalistic pedophiles operate a global child sex trafficking ring and conspired against former President Donald Trump during his term in office. QAnon has been described as a cult.'}

In [24]:
queriesWithIPs_audit['Chemtrails']

{'title': 'Contrail',
 'source': 'Encyclopedia Britannica',
 'url': 'https://www.britannica.com/science/vapor-trail',
 'text': 'Contrail, also called condensation trail or vapour trail, streamer of cloud sometimes observed behind an airplane flying in clear cold humid air. It forms upon condensation of water vapour produced by the combustion of fuel in airplane engines. When the ambient relative humidity is high, the resulting ice-crystal plume may last several hours. The trail may be distorted by the winds, and sometimes it spreads outward to form a layer of cirrus cloud. On rare occasions, when the air is nearly saturated with water vapour, air circulation at the wing tips of an airplane may cause sufficient pressure and temperature reductions to cause cloud streamers to form. In the 1990s a popular conspiracy theory arose claiming that long-lasting contrails contained chemicals (“chemtrails”) that were sprayed by governments for purposes such as controlling the weather or dispersing

In [25]:
queriesWithIPs['russia shot down mh370']

{'title': 'Malaysia Airlines flight 370 disappearance',
 'source': 'Encyclopedia Britannica',
 'url': 'https://www.britannica.com/event/Malaysia-Airlines-flight-370-disappearance',
 'text': 'Malaysia Airlines flight 370 disappearance, also called MH370 disappearance, disappearance of a Malaysia Airlines passenger jet on March 8, 2014, during a flight from Kuala Lumpur to Beijing. The disappearance of the Boeing 777 with 227 passengers and 12 crew members on board led to a search effort stretching from the Indian Ocean west of Australia to Central Asia. Flight 370 took off at 12:41 AM local time. It reached a cruising altitude of 10,700 metres (35,000 feet) at 1:01 AM. The Aircraft Communication Addressing and Reporting System (ACARS), which transmitted data about the aircraft’s performance, sent its last transmission at 1:07 AM and was subsequently switched off. The last voice communication from the crew occurred at 1:19 AM, and at 1:21 AM the plane’s transponder, which communicated wi

**Save the data**

It's always good to save partial results of the analysis, so that one doesn't need to repeat it again.

In [26]:
with open('onlyQueriesWithIPs.json', 'w') as outputF:
    json.dump(queriesWithIPs, outputF)

In [27]:
with open('onlyQueriesWithIPs_audit.json', 'w') as outputM:
    json.dump(queriesWithIPs_audit, outputM)

## Step 2: Extract IPs from Watch pages

First, extract the list of videos from all SERP pages using the main function `extractVideosFromSERP`.
Once extracted, store them in a dictionary of dictionaries for further processing.

In [28]:
files = os.listdir('SERPS-crowd') 
files = [f for f in files if not f.startswith('.')]

queriesWithVideos = {}
for f in files:
    query = f.split('.html')[0]
    result =  extractVideosFromSERP(os.path.join('SERPS-crowd', f))
    queriesWithVideos[query] = extractVideosFromSERP(os.path.join('SERPS-crowd', f))

# Print query, total number of videos, and the first video info
for query in queriesWithVideos:
    print(query, len(queriesWithVideos[query]), queriesWithVideos[query][0])

crisis actors 20 {'videoId': '1w03kVRTYI8', 'title': 'Inside the ‘Crisis Actor’ Conspiracy Theory'}
school shooting crisis actors 20 {'videoId': 'To91BJGKr5I', 'title': 'The Rise of the Crisis Actor Conspiracy Movement'}
obama birth 20 {'videoId': 'QM2GJn6hpJE', 'title': 'President Obama Provides His Birth Certificate to the Press'}
moon landing faked 20 {'videoId': 'uTChrirK-hw', 'title': 'Was the Moon Landing faked? | Big Questions with Neil deGrasse Tyson'}
jfk killer 20 {'videoId': 'sjXRQXlBU3g', 'title': 'Lee Harvey Oswald Shot On Camera | Archives | TODAY'}
vaccines autism 20 {'videoId': 'o65l1YAVaYc', 'title': "Vaccines Don't Cause Autism: Healthcare Triage #12"}
government poisoning 20 {'videoId': 'CWo8vNRbPyQ', 'title': "Yes, The U.S. Gov't Poisoned Alcohol in the 1920s"}
stevie wonder faked blindness 20 {'videoId': 'jzVXkbvauWY', 'title': "Stevie Wonder Isn't Blind?"}
alaska mind control lab 20 {'videoId': 'BoZf9feQATc', 'title': 'The Truth Behind This Big Alaskan Conspiracy 

In [131]:
files_audit = os.listdir('SERPS-audit') 
files_audit = [m for m in files_audit if not m.startswith('.')]

queriesWithVideos_audit = {}
for m in files_audit:
    query_audit = m.split('.html')[0]
    result_audit =  extractVideosFromSERP(os.path.join('SERPS-audit', m))
    queriesWithVideos_audit[query_audit] = extractVideosFromSERP(os.path.join('SERPS-audit', m))

# Print query, total number of videos, and the first video info
for query_audit in queriesWithVideos_audit:
    print(query_audit, len(queriesWithVideos_audit[query_audit]), queriesWithVideos_audit[query_audit][0])

chemtrail whistleblower 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
anti vaccine 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
911 conspiracy shane 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
chemtrail plane up close 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
911 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
moon hoax 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
Chemtrail flu 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Planet America'}
911 footage ground zero 20 {'videoId': 'gGsUUvOXxtY', 'title': 'Roswell: The UFO mystery that still haunts America | Pl

**Save results**

It's good to save these results for future reference, and also to avoid running the above code.

In [30]:
with open('SERP-crowd-videos-list.json', 'w') as outputF:
    json.dump(queriesWithVideos, outputF)

In [89]:
with open('SERP-audit-videos-list.json', 'w') as outputM:
    json.dump(queriesWithVideos_audit, outputM)

### Gather some Watch pages to test IP extraction

To test the code I wrote for extracting IP info, I need to retrieve some Watch pages first.
I will do it for the "flat earth proof" videos, since I know that many of those videos have IPs.

**Note:** In the current folder, I have a subfolder Watch-crowd to store the videos I'm collecting below.

### Extract for crowd file

In [68]:
#get watch pages for all the queries in crowd
for i in queries:
    VL_crowd = queriesWithVideos[i]
    for video in VL_crowd:
        videoId = video['videoId']
        url = 'https://www.youtube.com/watch?v=' + videoId
        respose = requests.get(url)
        if response.status_code == 200:
            text = response.text
            with open(os.path.join(fldPath_crowd, "{}.html".format(videoId)), 'w') as outputF:
                outputF.write(text)    
        fldPath_crowd = 'Watch-crowd-1' # folder to store the files

### Extract for audit file

In [133]:
#get watch pages for all the queries in crowd
for i in queries_audit:
    VL_audit = queriesWithVideos_audit[i]
    for video in VL_audit:
        videoId = video['videoId']
        url = 'https://www.youtube.com/watch?v=' + videoId
        response_audit = requests.get(url)
        if response_audit.status_code == 200:
            text_audit = response_audit.text
            with open(os.path.join(fldPath_audit, "{}.html".format(videoId)), 'w') as outputM:
                outputM.write(text_audit)
        fldPath_audit = 'Watch-audit-1' # folder to store the files

In [117]:
len(queries_audit)

49

## Extract for a specific query and test it out 

In [36]:
flatEarthVL = queriesWithVideos['flat earth proof']
fldPath = 'Watch-crowd' # folder to store the files

In [37]:
#flat earth crowd
for video in flatEarthVL:
    videoId = video['videoId']
    url = 'https://www.youtube.com/watch?v=' + videoId
    response = requests.get(url)
    if response.status_code == 200:
        text = response.text
        with open(os.path.join(fldPath, "{}.html".format(videoId)), 'w') as outputF:
            outputF.write(text)

In [128]:
flatEarthVL_audit = queriesWithVideos_audit['flat earth proof']
fldPath_audit1 = 'Watch-crowd-audit' # folder to store the files

In [129]:
#flat earth audit
for video in flatEarthVL_audit:
    videoId = video['videoId']
    url = 'https://www.youtube.com/watch?v=' + videoId
    response_audit = requests.get(url)
    if response_audit.status_code == 200:
        text_audit = response_audit.text
        with open(os.path.join(fldPath_audit1, "{}.html".format(videoId)), 'w') as outputM:
            outputM.write(text_audit)

If I check the Watch-crowd folder, I can see the files stored there.

### Check Watch pages for IP and extract it if present

In [71]:
files = os.listdir(fldPath_crowd)

watchPagesIPdict = {} # store the IP information here

# call the function extractIP on every video
for f in files:
    vId = f.split('.html')[0]
    filePath = os.path.join(fldPath_crowd, f)
    watchPagesIPdict[vId] = extractIP(filePath, 'Watch')
    

# how many videos have a non-empty IP dict?
withIPs = [vId for vId in watchPagesIPdict if watchPagesIPdict[vId]]
len(withIPs)

1138

In [119]:
files_audit = os.listdir(fldPath_audit1)

watchPagesIPdict_audit = {} # store the IP information here

# call the function extractIP on every video
for m in files_audit:
    vId_audit = m.split('.html')[0]
    filePath_audit = os.path.join(fldPath_audit1, m)
    watchPagesIPdict_audit[vId_audit] = extractIP(filePath_audit, 'Watch')
    

# how many videos have a non-empty IP dict?
withIPs_audit = [vId_audit for vId_audit in watchPagesIPdict_audit if watchPagesIPdict_audit[vId_audit]]
len(withIPs_audit)

5

What is stored in one dictionary?

In [80]:
watchPagesIPdict_audit.keys()

dict_keys(['1waEzUA5Mek', 'gGsUUvOXxtY', 'odUSnDgU-oo', 'EWZWVEkqVS8', 'CtrR84jboT4', 'o1RS7D_szJo', '9-8xTPilki4', 'lD3gETJeCkg', 'Qnym-ZcckYM', 'foJWblpzEjA', 'MSkTLwqBA3w', 'w8ikEOair-Q', 'Tim1rqGsUxc', 'lvimRsH1O_4', '1lDx1tx0tSI', '7OW74On3sAU', 'FqB0BKOdm_E', 'DH4UskC7WQA', 'dzv6nRpMX24', 'tF4m8SuX5Gc'])

In [126]:
len(watchPagesIPdict)

1138

In [43]:
watchPagesIPdict_audit[withIPs_audit[6]]

{'title': 'Area 51',
 'source': 'Encyclopedia Britannica',
 'url': 'https://www.britannica.com/place/Area-51',
 'text': 'Area 51, secret U.S. Air Force military installation located at Groom Lake in southern Nevada. It is administered by Edwards Air Force Base in southern California. The installation has been the focus of numerous conspiracies involving extraterrestrial life, though its only confirmed use is as a flight testing facility. For years there was speculation about the installation, especially amid growing reports of UFO sightings in the vicinity. The site became known as Area 51, which was its designation on maps of the Atomic Energy Commission. Conspiracy theories gained support in the late 1980s, when a man alleging to have worked at the installation claimed that the government was examining recovered alien spacecraft.'}

### Combine information to create a dataframe

The cell below only shows how to create the dataframe for a single query. When you have a folder that has the Watch files for all the queries, you can use this code within a for loop that iterates over all query names. Then, you can create a larger dataframe with information about all videos.

In [44]:
queryName = 'flat earth proof'

ipTitle = queriesWithIPs.get(queryName, {}).get('title','') # the title of the IP panel, if there is one for the query


dataForDF = []
for videoDct in queriesWithVideos[queryName]:
    # update videoDct with info about the SERP IP
    videoDct.update({'query': queryName, 
                     'IPTitleSERP': ipTitle, 
                     'hasSERP_IP': (ipTitle != '') # generate a boolean value if it has an IP title
                    }) 
    
    # update videoDct with info about the Watch IP
    videoIP = watchPagesIPdict[videoDct['videoId']]
    
    ipTitleWatch = videoIP.get('title', '')
    videoDct.update({'IPTitleWatch': ipTitleWatch,
                    'hasWatch_IP': (ipTitleWatch != '')})
    
    dataForDF.append(videoDct)

In [122]:
dataForDF1 = []

#ipTitle = queriesWithIPs.get(queryName, {}).get('title','') # the title of the IP panel, if there is one for the query

for queryName in queries:
    #ipTitle = queriesWithIPs.get(queryName, {}).get('title','') 
    for videoDct in queriesWithVideos[queryName]:
        ipTitle = queriesWithIPs.get(queryName, {}).get('title','')
        # update videoDct with info about the SERP IP
        videoDct.update({'query': queryName, 
                            'IPTitleSERP': ipTitle, 
                            'hasSERP_IP': (ipTitle != '') # generate a boolean value if it has an IP title
                            }) 
        dataForDF1.append(videoDct)
        # update videoDct with info about the Watch IP
        videoIP = watchPagesIPdict[videoDct['videoId']]
    
        ipTitleWatch = videoIP.get('title', '')
        videoDct.update({'IPTitleWatch': ipTitleWatch,
                        'hasWatch_IP': (ipTitleWatch != '')})
    #.unique()
         

In [137]:
#watchPagesIPdict.keys()

Let's check one entry:

In [98]:
dataForDF[0]

{'videoId': 'e5ACN9iF8Jw',
 'title': '5 Facts That Prove The Earth Is Flat',
 'query': 'flat earth proof',
 'IPTitleSERP': 'Flat Earth',
 'hasSERP_IP': True,
 'IPTitleWatch': 'Flat Earth',
 'hasWatch_IP': True}

We can now use this list of dictionaries to create a dataframe in pandas:

In [47]:
import pandas as pd

In [123]:
df1 = pd.DataFrame(dataForDF1)
df1.head(450)

,videoId,title,query,IPTitleSERP,hasSERP_IP,IPTitleWatch,hasWatch_IP
0,e5ACN9iF8Jw,5 Facts That Prove The Earth Is Flat,flat earth proof,Flat Earth,True,Flat Earth,True
1,06bvdFK3vVU,Flat Earth vs. Round Earth | Explorer,flat earth proof,Flat Earth,True,Flat Earth,True
2,su-fmoZUkF8,Destroying Flat Earth Without Using Science - ...,flat earth proof,Flat Earth,True,Flat Earth,True
3,1gHbwT_R9t0,"Inside a Flat Earth convention, where nearly e...",flat earth proof,Flat Earth,True,Flat Earth,True
4,thxbiR-XfJo,An Astronomer Responds To Flat Earth Theory,flat earth proof,Flat Earth,True,Flat Earth,True
...,...,...,...,...,...,...,...
445,yqgMECkW3Ak,Donald Trump Believes Climate Change Is A Hoax...,climate change fake,Climate change,True,Flat Earth,True
446,UmIJCGQzCiU,Conversation with global warming skeptic Antho...,climate change fake,Climate change,True,Flat Earth,True
447,y2euBvdP28c,Why People Don't Believe In Climate Science,climate change fake,Climate change,True,Flat Earth,True
448,f1rxv1yPQrc,Why Climate Change Denial Still Exists In The ...,climate change fake,Climate change,True,Flat Earth,True


In [135]:
df1.to_csv('Crowd-all_queries.csv')

In [49]:
df = pd.DataFrame(dataForDF)
df

,videoId,title,query,IPTitleSERP,hasSERP_IP,IPTitleWatch,hasWatch_IP
0,e5ACN9iF8Jw,5 Facts That Prove The Earth Is Flat,flat earth proof,Flat Earth,True,Flat Earth,True
1,06bvdFK3vVU,Flat Earth vs. Round Earth | Explorer,flat earth proof,Flat Earth,True,Flat Earth,True
2,su-fmoZUkF8,Destroying Flat Earth Without Using Science - ...,flat earth proof,Flat Earth,True,Flat Earth,True
3,1gHbwT_R9t0,"Inside a Flat Earth convention, where nearly e...",flat earth proof,Flat Earth,True,Flat Earth,True
4,thxbiR-XfJo,An Astronomer Responds To Flat Earth Theory,flat earth proof,Flat Earth,True,Flat Earth,True
5,iwku5Alsi04,Man launches himself in self-made rocket to pr...,flat earth proof,Flat Earth,True,Flat Earth,True
6,xM9S4QiEHEY,Proving The Earth Is Flat!,flat earth proof,Flat Earth,True,Flat Earth,True
7,8HYMb3MgQRg,3 scientists school flat Earthers on the evide...,flat earth proof,Flat Earth,True,Flat Earth,True
8,Dd-FAyHdpxI,Destroying Flat Earth Without Using Science - ...,flat earth proof,Flat Earth,True,Flat Earth,True
9,DPRJzNc876A,This Flat Earther Thinks NASA is Lying to You,flat earth proof,Flat Earth,True,Flat Earth,True


In [50]:
dataForDF[1]

{'videoId': '06bvdFK3vVU',
 'title': 'Flat Earth vs. Round Earth | Explorer',
 'query': 'flat earth proof',
 'IPTitleSERP': 'Flat Earth',
 'hasSERP_IP': True,
 'IPTitleWatch': 'Flat Earth',
 'hasWatch_IP': True}

In [51]:
queryName = 'chemtrail pilot'

ipTitle = queriesWithIPs.get(queryName, {}).get('title','') # the title of the IP panel, if there is one for the query


dataForDF_audit = []
for videoDct in queriesWithVideos_audit[queryName]:
    # update videoDct with info about the SERP IP
    videoDct.update({'query': queryName, 
                     'IPTitleSERP': ipTitle, 
                     'hasSERP_IP': (ipTitle != '') # generate a boolean value if it has an IP title
                    }) 
    
    # update videoDct with info about the Watch IP
    videoIP = watchPagesIPdict_audit[videoDct['videoId']]
    
    ipTitleWatch = videoIP.get('title', '')
    videoDct.update({'IPTitleWatch': ipTitleWatch,
                    'hasWatch_IP': (ipTitleWatch != '')})
    
    dataForDF_audit.append(videoDct)

In [52]:
df2 = pd.DataFrame(dataForDF_audit)
df2

,videoId,title,query,IPTitleSERP,hasSERP_IP,IPTitleWatch,hasWatch_IP
0,gGsUUvOXxtY,Roswell: The UFO mystery that still haunts Ame...,chemtrail pilot,,False,,False
1,odUSnDgU-oo,The Mysterious Roswell UFO Incident of 1947,chemtrail pilot,,False,,False
2,foJWblpzEjA,Ask an Expert: The Roswell Incident,chemtrail pilot,,False,,False
3,MSkTLwqBA3w,Unlocking & Debunking The Roswell UFO Mystery ...,chemtrail pilot,,False,,False
4,EWZWVEkqVS8,"Trump Discusses Declassifying Roswell, Says He...",chemtrail pilot,,False,,False
5,Qnym-ZcckYM,"History behind supposed UFO crash in Roswell, ...",chemtrail pilot,,False,,False
6,o1RS7D_szJo,Roswell's Bizarre UFO Crash,chemtrail pilot,,False,Area 51,True
7,FqB0BKOdm_E,"""Roswell: The First Witness"" on the HISTORY ch...",chemtrail pilot,,False,,False
8,9-8xTPilki4,Behind Roswell: America's first UFO conspiracy...,chemtrail pilot,,False,Area 51,True
9,lD3gETJeCkg,UFO Mystery: The 65Th Anniversary Of Roswell |...,chemtrail pilot,,False,,False


In [124]:
dataForDF3 = []

#ipTitle = queriesWithIPs.get(queryName, {}).get('title','') # the title of the IP panel, if there is one for the query

for queryName in queries_audit:
    ipTitle = queriesWithIPs_audit.get(queryName, {}).get('title','')
    #ipTitle = queriesWithIPs[queryName].get('title','')
    for videoDct in queriesWithVideos_audit[queryName]:
        # update videoDct with info about the SERP IP
        videoDct.update({'query': queryName, 
                            'IPTitleSERP': ipTitle, 
                            'hasSERP_IP': (ipTitle != '') # generate a boolean value if it has an IP title
                            }) 
        dataForDF3.append(videoDct)
        # update videoDct with info about the Watch IP
        videoIP = watchPagesIPdict[videoDct['videoId']]
    
        ipTitleWatch = videoIP.get('title', '')
        videoDct.update({'IPTitleWatch': ipTitleWatch,
                        'hasWatch_IP': (ipTitleWatch != '')})
    #.unique()
         

In [125]:
df3 = pd.DataFrame(dataForDF3)
df3.head(70)

,videoId,title,query,IPTitleSERP,hasSERP_IP,IPTitleWatch,hasWatch_IP
0,gGsUUvOXxtY,Roswell: The UFO mystery that still haunts Ame...,911,,False,Flat Earth,True
1,odUSnDgU-oo,The Mysterious Roswell UFO Incident of 1947,911,,False,Flat Earth,True
2,foJWblpzEjA,Ask an Expert: The Roswell Incident,911,,False,Flat Earth,True
3,MSkTLwqBA3w,Unlocking & Debunking The Roswell UFO Mystery ...,911,,False,Flat Earth,True
4,EWZWVEkqVS8,"Trump Discusses Declassifying Roswell, Says He...",911,,False,Flat Earth,True
...,...,...,...,...,...,...,...
65,Qnym-ZcckYM,"History behind supposed UFO crash in Roswell, ...",911 calls from plane,,False,Flat Earth,True
66,o1RS7D_szJo,Roswell's Bizarre UFO Crash,911 calls from plane,,False,Flat Earth,True
67,FqB0BKOdm_E,"""Roswell: The First Witness"" on the HISTORY ch...",911 calls from plane,,False,Flat Earth,True
68,9-8xTPilki4,Behind Roswell: America's first UFO conspiracy...,911 calls from plane,,False,Flat Earth,True


In [136]:
df3.to_csv('audit-all_queries.csv')